This script evaluates the performance of different graph-aware architectures in a node classification problem. Several datasets are employed paying special attention to the homophily ratio.

In [1]:
import time
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import dgl
import networkx as nx
import torch
import torch.nn as nn


import utils
from gsp_utils.baselines_archs import GCNN_2L, MLP, GAT
from gsp_utils.baselines_models import NodeClassModel, GF_NodeClassModel
from gsp_utils.data import normalize_gso
from src.arch import GFGCN, GFGCNLayer, GFGCN_noh_Layer, GFGCN_Spows, Dual_GFGCN 

# SEED = 0
SEED = 15
PATH = 'results/try_datasets'
SAVE = False
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

torch.manual_seed(SEED)

cuda:0


In [2]:
def print_full_results(accs, ellapsed_time, datasets, exps):
    mean_accs = accs.mean(axis=2)
    med_accs = np.median(accs, axis=2)
    std_accs = accs.std(axis=2)
    mean_t = ellapsed_time.mean(axis=2)

    for i, dataset_name in enumerate(datasets):
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])

        print(f'{dataset_name} (homophily ratio: {edge_hom:.3f})')
        for j, exp in enumerate(exps):
            print(f'\t- {exp["leg"]}:\tmean: {mean_accs[j,i]:.3f} - std: {std_accs[j,i]:.4f} - med: {med_accs[j,i]:.3f} - time: {mean_t[j,i]:.2f} mins')
        
        print()

def summary_table(accs, datasets, exps, median=False):
    mean_accs = accs.mean(axis=2)
    cols_name = []
    for dataset_name in datasets:
        graph = getattr(dgl.data, dataset_name)(verbose=False)[0]
        edge_hom = dgl.edge_homophily(graph, graph.ndata['label'])
        cols_name.append(f'{dataset_name} ({edge_hom:.2f})')

    index_name = [exp['leg'] for exp in exps]

    return DataFrame(mean_accs, columns=cols_name, index=index_name)


In [3]:
DATASETS = ['TexasDataset',  'WisconsinDataset', 'CornellDataset', 'ChameleonDataset', 'CoraGraphDataset']
# DATASETS = ['CiteseerGraphDataset', 'CoraGraphDataset']


## Best params

In [4]:
N_RUNS = 10
ACT = nn.LeakyReLU()  # nn.ReLU()
LAST_ACT = nn.Softmax(dim=1)
LOSS_FN = nn.CrossEntropyLoss() #nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'L': 2, 'hid_dim': 16, 'leg': 'Kipf-A'},

        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'hid_dim': 16, 'leg': 'MLP'},
        {'model': 'MLP', 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': .5,
          'hid_dim': 32, 'leg': 'MLP'},

        {'model': 'GAT', 'heads': 2, 'epochs': 200, 'lr': .01, 'wd': 5e-4, 'drop': 0,
          'hid_dim': 16, 'leg': 'GAT'},
          

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 10, 'e_W': 10, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50, 'L': 3, 'K': 2, 'h0': 1, 'norm': True, 'leg': 'A-GCN-normA'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'A-GCN'},

        {'model': 'GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'hid_dim': 50,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'A-GCN-v2'},

        {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 5, 'e_W': 25, 'lr': .005, 'wd': 5e-4, 'drop': .5,
         'hid_dim': 32,'L': 3, 'K': 2, 'norm': True, 'leg': 'H-GCN-normH'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .5,
         'alpha': .5, 'hid_dim': 32,'L': 3, 'K': 2, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .5,
         'alpha': None, 'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-v2'},

        {'model': 'Dual-GFGCN', 'epochs': 200, 'e_h': 50, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
         'alpha': None, 'hid_dim': 32,'L': 2, 'K': 3, 'h0': 1, 'norm': False, 'leg': 'Dual-GFCN-v3'},

        # {'model': 'H-GFGCN', 'epochs': 200, 'e_h': 25, 'e_W': 25, 'lr': .005, 'wd': .001, 'drop': .25,
        #  'hid_dim': 32,'L': 2, 'K': 3, 'norm': False, 'leg': 'H-GCN'},
         

        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'epochs': 500, 'lr': .005, 'wd': .01, 'drop': .25,
         'hid_dim': 32, 'L': 2, 'K': 2, 'norm': False, 'leg': 'W-GCN'},
      ]


In [5]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, exp['hid_dim'], out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=exp['drop'], norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  exp['hid_dim'], out_dim, dropout=exp['drop'])

            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': exp['drop']}
                arch = GAT(in_dim,  exp['hid_dim'], out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=exp['drop'], diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=exp['drop'], norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=exp['drop'], diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'Dual-GFGCN':
                arch = Dual_GFGCN(in_dim, exp['hid_dim'], out_dim, exp['L'], exp['K'], act=ACT, alpha=exp['alpha'],
                                  last_act=LAST_ACT, dropout=exp['drop'], init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'])
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)
                loss, acc = model.train(feat, labels, exp['epochs'], exp['lr'], exp['wd'],
                                        epochs_h=exp['e_h'], epochs_W=exp['e_W'])
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc1 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val1 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val1b = summary_table(accs_best_val2, DATASETS, EXPS)

TexasDataset
0: 0.568 (0.676) - 0.703 (0.730) - 0.757 (0.784) - 0.811 (0.811) - 0.486 (0.649) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in true_divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.838 (0.838) - 0.838 (0.865) - 0.865 (0.919) - 0.838 (0.892) - 0.892 (0.919) - 0.865 (0.919) - 0.865 (0.892) - 0.811 (0.865) - 0.784 (0.865) - 
1: 0.541 (0.622) - 0.595 (0.595) - 0.757 (0.892) - 0.730 (0.892) - 0.514 (0.595) - 0.919 (0.946) - 0.919 (1.000) - 0.946 (1.000) - 0.973 (0.973) - 0.919 (0.973) - 0.865 (0.919) - 0.919 (0.973) - 0.811 (0.892) - 0.784 (0.892) - 
2: 0.514 (0.541) - 0.486 (0.568) - 0.676 (0.730) - 0.811 (0.838) - 0.514 (0.568) - 0.838 (0.865) - 0.838 (0.919) - 0.784 (0.892) - 0.811 (0.919) - 0.784 (0.892) - 0.811 (0.892) - 0.811 (0.892) - 0.757 (0.757) - 0.757 (0.811) - 
3: 0.541 (0.649) - 0.649 (0.703) - 0.838 (0.892) - 0.784 (0.865) - 0.432 (0.622) - 0.784 (0.946) - 0.892 (0.919) - 0.919 (0.919) - 0.811 (0.919) - 0.892 (0.946) - 0.838 (0.892) - 0.838 (0.973) - 0.838 (0.919) - 0.784 (0.811) - 
4: 0.541 (0.595) - 0.622 (0.676) - 0.730 (0.730) - 0.730 (0.838) - 0.595 (0.622) - 0.784 (0.865) - 0.730 (0.892) - 0.865 (0.892) - 0.838 (0.865) - 0.865 (0.973) - 0.757 (0

In [6]:
if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_acc_val1.to_csv(PATH + 'best_params_val_test_' + timestr)
    table_acc_val1.to_csv(PATH + 'best_params_best_val_' + timestr)

In [7]:
table_acc_val1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81)
Kipf-normA,0.583784,0.562745,0.456757,0.616228,0.5288
Kipf-A,0.618919,0.474510,0.435135,0.348246,0.7747
MLP,0.759459,0.829412,0.732432,0.460307,0.5671
MLP,0.751351,0.831373,0.751351,0.470833,0.5974
GAT,0.570270,0.500000,0.432432,0.622588,0.5951
A-GCN-normA,0.832432,0.870588,0.772973,0.467763,0.7954
A-GCN,0.835135,0.870588,0.797297,0.445395,0.7507
A-GCN-v2,0.843243,0.858824,0.783784,0.459868,0.7595
H-GCN-normH,0.845946,0.864706,0.767568,0.427851,0.7950
Dual-GFCN,0.856757,0.854902,0.772973,0.523246,0.7781


In [8]:
table_acc_val1b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81)
Kipf-normA,0.578378,0.574510,0.454054,0.611842,0.5278
Kipf-A,0.621622,0.464706,0.440541,0.348904,0.7769
MLP,0.740541,0.817647,0.721622,0.489912,0.5553
MLP,0.762162,0.829412,0.740541,0.493860,0.5879
GAT,0.567568,0.519608,0.424324,0.620395,0.6053
A-GCN-normA,0.837838,0.862745,0.775676,0.469737,0.7935
A-GCN,0.851351,0.876471,0.781081,0.446491,0.7502
A-GCN-v2,0.840541,0.866667,0.778378,0.446711,0.7626
H-GCN-normH,0.851351,0.860784,0.778378,0.425658,0.7955
Dual-GFCN,0.840541,0.847059,0.759459,0.520833,0.7766


In [9]:
table_acc1

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CoraGraphDataset (0.81)
Kipf-normA,0.637838,0.615686,0.529730,0.624781,0.5325
Kipf-A,0.670270,0.496078,0.486486,0.361842,0.7874
MLP,0.805405,0.852941,0.770270,0.507895,0.5698
MLP,0.829730,0.872549,0.778378,0.503728,0.6011
GAT,0.645946,0.578431,0.491892,0.631579,0.6158
A-GCN-normA,0.894595,0.913725,0.848649,0.503947,0.8058
A-GCN,0.910811,0.917647,0.864865,0.463158,0.7701
A-GCN-v2,0.910811,0.923529,0.878378,0.481360,0.7826
H-GCN-normH,0.908108,0.905882,0.832432,0.449561,0.8002
Dual-GFCN,0.929730,0.900000,0.843243,0.549342,0.7889


## Initial Params

In [ ]:
# BEST PARAMETERS
## Reaining params
N_RUNS = 10
N_EPOCHS = 200  # 500
LR = .01
WD = 5e-4
DROPOUT = .5

# BEST PARAMETERS
## Architecture params
N_LAYERS = 2
K = 3
HID_DIM = 16

## Model params
h0 = 1
ACT = nn.ReLU()
LAST_ACT = nn.LogSoftmax(dim=1)
LOSS_FN = nn.NLLLoss()

EXPS = [{'model': 'Kipf', 'norm': 'both', 'leg': 'Kipf-normA'},
        {'model': 'Kipf', 'norm': 'none', 'leg': 'Kipf-A'},
        {'model': 'MLP', 'leg': 'MLP-conf1'},
        {'model': 'GAT', 'heads': 2, 'hid_dim': 16, 'leg': 'GAT'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': True, 'leg': 'A-GCN-normA'},
        {'model': 'GFGCN', 'L': N_LAYERS, 'K': K, 'h0': h0, 'norm': False, 'leg': 'A-GCN'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'H-GCN-normH'},
        {'model': 'H-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'GCN-H'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': True, 'leg': 'W-GCN-normA'},
        {'model': 'noh-GFGCN', 'L': N_LAYERS, 'K': K, 'norm': False, 'leg': 'W-GCN'},]

In [ ]:
best_accs = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
accs_best_val2 = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
ellapsed_times = np.zeros((len(EXPS), len(DATASETS), N_RUNS))
for j, dataset in enumerate(DATASETS):

    print(dataset)

    for i in range(N_RUNS):
        print(f'{i}:', end=' ')
        
        A, feat, labels, n_class, masks = utils.get_data_dgl(dataset, dev=device, idx=i%10)
        N = A.shape[0]
        in_dim = feat.shape[1]
        out_dim = n_class
        
        for k, exp in enumerate(EXPS):
            t_i = time.time()
            if exp['model'] == 'Kipf':
                arch = GCNN_2L(in_dim, HID_DIM, out_dim, act=ACT, last_act=LAST_ACT,
                               dropout=DROPOUT, norm=exp['norm'])
                S = dgl.from_networkx(nx.from_numpy_array(A)).add_self_loop().to(device)

            elif exp['model'] == 'MLP':
                arch = MLP(in_dim,  HID_DIM, out_dim, dropout=DROPOUT, act=ACT, last_act=LAST_ACT)
            
            elif exp['model'] == 'GAT':
                gat_params = {'attn_drop': DROPOUT}
                arch = GAT(in_dim,  HID_DIM, out_dim, exp['heads'], gat_params, act=ACT, last_act=LAST_ACT)

            elif exp['model'] == 'GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                         dropout=DROPOUT, diff_layer=GFGCNLayer, init_h0=exp['h0'])
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)

            elif exp['model'] == 'H-GFGCN':
                arch = GFGCN_Spows(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                                   dropout=DROPOUT, norm=exp['norm'], dev=device)
                S = torch.Tensor(A).to(device)

            elif exp['model'] == 'noh-GFGCN':
                arch = GFGCN(in_dim, HID_DIM, out_dim, exp['L'], exp['K'], act=ACT, last_act=LAST_ACT,
                             dropout=DROPOUT, diff_layer=GFGCN_noh_Layer)
                if exp['norm']:
                    S = torch.Tensor(normalize_gso(A, 'both')).to(device)
                else:
                    S = torch.Tensor(A).to(device)
            else:
                raise Exception(f'ERROR: unknown architecture {exp["model"]}')

            if exp['model'] in ['Kipf', 'MLP', 'GAT', 'noh-GFGCN']:
                model = NodeClassModel(arch, S, masks, LOSS_FN, device=device)
            else:
                model = GF_NodeClassModel(arch, S, exp['K'], masks, LOSS_FN, device=device)

            loss, acc = model.train(feat, labels, N_EPOCHS, LR, WD)
            ellapsed_t = (time.time()-t_i)/60
            
            
            best_accs[k,j,i] = np.max(acc["test"])
            accs_best_val[k,j,i] = model.test(feat, model.S, labels, masks['test'])
            accs_best_val2[k,j,i] = acc["test"][np.argmax(acc["val"])]

            ellapsed_times[k,j,i] = ellapsed_t

            print(f'{accs_best_val[k,j,i]:.3f} ({best_accs[k,j,i]:.3f})', end=' - ')
        print()      
    print()


print_full_results(accs_best_val, ellapsed_times, DATASETS, EXPS)
table_acc2 = summary_table(best_accs, DATASETS, EXPS)
table_acc_val2 = summary_table(accs_best_val, DATASETS, EXPS)
table_acc_val2b = summary_table(accs_best_val2, DATASETS, EXPS)

if SAVE:
    timestr = time.strftime("%Y%m%d-%H%M")
    table_acc_val2.to_csv(PATH + 'init_params_val_test_' + timestr)
    table_acc_val2.to_csv(PATH + 'init_params_best_val_' + timestr)

TexasDataset
0: 0.649 (0.649) - 0.676 (0.730) - 0.784 (0.784) - 0.622 (0.649) - 

/home/srey/Investigacion/robust_minmax_gnn/gsp_utils/data.py:44: RuntimeWarning: divide by zero encountered in true_divide
  D_inv = np.diag(np.where(np.isclose(deg_vec, 0), 0, 1/deg_vec))


0.351 (0.514) - 0.541 (0.541) - 0.568 (0.649) - 0.568 (0.595) - 0.811 (0.838) - 0.784 (0.784) - 
1: 0.568 (0.595) - 0.676 (0.703) - 0.757 (0.757) - 0.595 (0.622) - 0.486 (0.649) - 0.622 (0.622) - 0.541 (0.622) - 0.595 (0.622) - 0.649 (0.757) - 0.649 (0.676) - 
2: 0.514 (0.541) - 0.459 (0.514) - 0.730 (0.757) - 0.514 (0.514) - 0.514 (0.595) - 0.514 (0.541) - 0.270 (0.486) - 0.514 (0.541) - 0.649 (0.730) - 0.378 (0.541) - 
3: 0.595 (0.649) - 0.622 (0.622) - 0.838 (0.865) - 0.486 (0.649) - 0.595 (0.622) - 0.541 (0.568) - 0.514 (0.514) - 0.486 (0.595) - 0.811 (0.865) - 0.595 (0.703) - 
4: 0.514 (0.568) - 0.568 (0.568) - 0.811 (0.865) - 0.568 (0.595) - 0.432 (0.541) - 0.486 (0.595) - 0.514 (0.568) - 0.486 (0.514) - 0.649 (0.730) - 0.514 (0.622) - 
5: 0.568 (0.622) - 0.514 (0.595) - 0.730 (0.865) - 0.568 (0.649) - 0.514 (0.595) - 0.568 (0.568) - 0.514 (0.622) - 0.622 (0.676) - 0.703 (0.865) - 0.622 (0.676) - 
6: 0.541 (0.541) - 0.568 (0.568) - 0.649 (0.703) - 0.541 (0.568) - 0.568 (0.595) - 

In [ ]:
table_acc_val2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.583784,0.531373,0.427027,0.629825,0.7145,0.8117
Kipf-A,0.591892,0.445098,0.370270,0.270175,0.6624,0.7683
MLP-conf1,0.751351,0.829412,0.721622,0.469737,0.5654,0.5760
GAT,0.562162,0.541176,0.440541,0.624781,0.7142,0.8086
A-GCN-normA,0.494595,0.482353,0.421622,0.377632,0.6363,0.7846
A-GCN,0.521622,0.425490,0.348649,0.217982,0.4838,0.6267
H-GCN-normH,0.443243,0.470588,0.335135,0.327412,0.7042,0.8129
GCN-H,0.554054,0.466667,0.354054,0.227851,0.4854,0.5788
W-GCN-normA,0.710811,0.700000,0.618919,0.485526,0.7086,0.8178
W-GCN,0.605405,0.566667,0.518919,0.228070,0.4986,0.5458


In [ ]:
table_acc_val2b

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.586486,0.515686,0.440541,0.638377,0.7117,0.8087
Kipf-A,0.600000,0.500000,0.429730,0.319298,0.6665,0.7691
MLP-conf1,0.754054,0.837255,0.735135,0.485746,0.5626,0.5697
GAT,0.570270,0.511765,0.427027,0.645395,0.7078,0.8121
A-GCN-normA,0.556757,0.490196,0.472973,0.449123,0.6754,0.7977
A-GCN,0.510811,0.450980,0.405405,0.247368,0.5026,0.6278
H-GCN-normH,0.527027,0.480392,0.402703,0.333991,0.7030,0.8136
GCN-H,0.537838,0.456863,0.402703,0.257456,0.5052,0.6163
W-GCN-normA,0.748649,0.760784,0.686486,0.511404,0.7058,0.8173
W-GCN,0.602703,0.592157,0.610811,0.277412,0.5625,0.6663


In [ ]:
table_acc2

,TexasDataset (0.11),WisconsinDataset (0.20),CornellDataset (0.13),ChameleonDataset (0.24),CiteseerGraphDataset (0.74),CoraGraphDataset (0.81)
Kipf-normA,0.613514,0.592157,0.527027,0.651535,0.7192,0.8185
Kipf-A,0.621622,0.527451,0.454054,0.326535,0.6733,0.7802
MLP-conf1,0.794595,0.858824,0.767568,0.503947,0.5689,0.5804
GAT,0.613514,0.605882,0.500000,0.660526,0.7253,0.8218
A-GCN-normA,0.605405,0.554902,0.529730,0.459430,0.6799,0.8068
A-GCN,0.583784,0.492157,0.451351,0.267105,0.5117,0.6375
H-GCN-normH,0.575676,0.550980,0.462162,0.350000,0.7151,0.8213
GCN-H,0.591892,0.523529,0.467568,0.264035,0.5146,0.6269
W-GCN-normA,0.789189,0.788235,0.727027,0.524123,0.7176,0.8250
W-GCN,0.678378,0.652941,0.651351,0.287939,0.5713,0.6728
